In [1]:
from flask import Flask, request
from flask_restful import Resource, Api
import pymysql
import pandas as pd

In [2]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='tutorcatdb'
)

# Define the SQL query to retrieve data from a table
query = 'SELECT * FROM ai_table;'

# Use pandas to load the query results into a DataFrame
ai_df = pd.read_sql(query, connection)

connection.close()

# Print the DataFrame
print(ai_df.head())

   NotesID  NotesID2  Val
0       25        25  1.0
1       26        25 -0.5
2       27        25  0.5
3       28        25 -0.5
4       29        25  1.0


In [3]:
piv_dataFrame = ai_df.pivot_table(index='NotesID', columns='NotesID2', values="Val").fillna(0)

piv_dataFrame

NotesID2,25,26,27,28,29,30,31,32,33,34
NotesID,,,,,,,,,,
25,1.000000,-0.500000,5.000000e-01,-5.000000e-01,1.000000,-5.000000e-01,0.981981,8.660250e-01,0.500000,-1.000000
26,-0.500000,1.000000,5.000000e-01,-5.000000e-01,-0.500000,-5.000000e-01,-0.654654,-8.660250e-01,-1.000000,0.500000
27,0.500000,0.500000,1.000000e+00,-1.000000e+00,0.500000,-1.000000e+00,0.327327,8.255430e-18,-0.500000,-0.500000
28,-0.500000,-0.500000,-1.000000e+00,1.000000e+00,-0.500000,1.000000e+00,-0.327327,2.426230e-17,0.500000,0.500000
29,1.000000,-0.500000,5.000000e-01,-5.000000e-01,1.000000,-5.000000e-01,0.981981,8.660250e-01,0.500000,-1.000000
30,-0.500000,-0.500000,-1.000000e+00,1.000000e+00,-0.500000,1.000000e+00,-0.327327,-1.520800e-18,0.500000,0.500000
31,0.981981,-0.654654,3.273270e-01,-3.273270e-01,0.981981,-3.273270e-01,1.000000,9.449110e-01,0.654654,-0.981981
32,0.866025,-0.866025,8.255430e-18,2.426230e-17,0.866025,-1.520800e-18,0.944911,1.000000e+00,0.866025,-0.866025
33,0.500000,-1.000000,-5.000000e-01,5.000000e-01,0.500000,5.000000e-01,0.654654,8.660250e-01,1.000000,-0.500000


In [4]:
def get_similarity_notes(notesID, user_rating):
    similarity_mark = piv_dataFrame[notesID] * user_rating
    similarity_mark = similarity_mark.sort_values(ascending=False)

    return similarity_mark

print(get_similarity_notes(25, 5))

NotesID
25    5.000000
29    5.000000
31    4.909905
32    4.330125
27    2.500000
33    2.500000
26   -2.500000
28   -2.500000
30   -2.500000
34   -5.000000
Name: 25, dtype: float64


In [5]:
notes_lover = [(25, 1), (26, 1), (27, 1)]

# piv_dataFrame_null = pd.DataFrame()
#
# for notesID,rating in notes_lover:
#     piv_dataFrame_null = piv_dataFrame.append(get_similarity_notes(notesID,rating),ignore_index=True)
#
# piv_dataFrame_null.head()
#
# piv_dataFrame_null.sum().sort_values(ascending = False)

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)


@app.route('/recommend/<int:student_id>', methods=['GET'])
def recommend(student_id):
    # req_data = request.get_json()
    # student_id = req_data['student_id']

    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='',
        db='tutorcatdb'
    )

    query = "SELECT NotesID,Rating FROM notes_rating WHERE StudentID={} ORDER BY Rating;".format(student_id)

    student_rating_df = pd.read_sql(query, connection)

    # Close the database connection
    connection.close()

    notes_id_list = [(row['NotesID'], 5 - row['Rating']) for _, row in student_rating_df.iterrows()]
    print(notes_id_list)
    print(type(notes_id_list))

    piv_Series_null = pd.DataFrame()

    for notesID,rating in notes_lover:
        piv_Series_null = piv_dataFrame.append(get_similarity_notes(notesID, rating), ignore_index=True)

    piv_Series_null.head()

    piv_Series_null= piv_Series_null.sum().sort_values(ascending = False)
    print(type(piv_Series_null))
    print(piv_Series_null)
    piv_dataFrame_null = piv_Series_null.to_frame(name='Val').rename_axis('NotesID')

    print(type(piv_dataFrame_null))
    print(piv_dataFrame_null)

    onlyIDlist = set([x[0] for x in notes_id_list])
    print(onlyIDlist)


    piv_dataFrame_null = piv_dataFrame_null[~piv_dataFrame_null.index.isin(onlyIDlist)]

    print(piv_dataFrame_null)

    notes_ids_list = piv_dataFrame_null.index.tolist()
    json_str = jsonify(notes_ids_list)
    # recommendations = ['Product 1', 'Product 2', 'Product 3']
    # return {'recommendations': recommendations}

    return json_str

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2023 22:19:16] "GET /recommend/10003 HTTP/1.0" 200 -


[(28, 3), (26, 2), (30, 2), (32, 2), (25, 1), (27, 1), (31, 1), (29, 0)]
<class 'list'>
<class 'pandas.core.series.Series'>
31    2.926892
25    2.848006
29    2.848006
32    2.810936
33    2.020679
27    0.827327
28   -0.827327
30   -0.827327
26   -2.020679
34   -2.848006
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
31       2.926892
25       2.848006
29       2.848006
32       2.810936
33       2.020679
27       0.827327
28      -0.827327
30      -0.827327
26      -2.020679
34      -2.848006
{32, 25, 26, 27, 28, 29, 30, 31}
              Val
NotesID          
33       2.020679
34      -2.848006


127.0.0.1 - - [22/Apr/2023 22:19:26] "GET /recommend/10003 HTTP/1.0" 200 -


[(28, 3), (26, 2), (30, 2), (32, 2), (25, 1), (27, 1), (31, 1), (29, 0)]
<class 'list'>
<class 'pandas.core.series.Series'>
31    2.926892
25    2.848006
29    2.848006
32    2.810936
33    2.020679
27    0.827327
28   -0.827327
30   -0.827327
26   -2.020679
34   -2.848006
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
31       2.926892
25       2.848006
29       2.848006
32       2.810936
33       2.020679
27       0.827327
28      -0.827327
30      -0.827327
26      -2.020679
34      -2.848006
{32, 25, 26, 27, 28, 29, 30, 31}
              Val
NotesID          
33       2.020679
34      -2.848006


127.0.0.1 - - [22/Apr/2023 22:33:30] "GET /recommend/10003 HTTP/1.0" 200 -


[(28, 3), (26, 2), (30, 2), (25, 1), (27, 1), (31, 1), (29, 0)]
<class 'list'>
<class 'pandas.core.series.Series'>
31    2.926892
25    2.848006
29    2.848006
32    2.810936
33    2.020679
27    0.827327
28   -0.827327
30   -0.827327
26   -2.020679
34   -2.848006
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
31       2.926892
25       2.848006
29       2.848006
32       2.810936
33       2.020679
27       0.827327
28      -0.827327
30      -0.827327
26      -2.020679
34      -2.848006
{25, 26, 27, 28, 29, 30, 31}
              Val
NotesID          
32       2.810936
33       2.020679
34      -2.848006


127.0.0.1 - - [22/Apr/2023 22:33:51] "GET /recommend/10003 HTTP/1.0" 200 -


[(28, 3), (26, 2), (30, 2), (25, 1), (27, 1), (31, 1), (32, 1), (29, 0)]
<class 'list'>
<class 'pandas.core.series.Series'>
31    2.926892
25    2.848006
29    2.848006
32    2.810936
33    2.020679
27    0.827327
28   -0.827327
30   -0.827327
26   -2.020679
34   -2.848006
dtype: float64
<class 'pandas.core.frame.DataFrame'>
              Val
NotesID          
31       2.926892
25       2.848006
29       2.848006
32       2.810936
33       2.020679
27       0.827327
28      -0.827327
30      -0.827327
26      -2.020679
34      -2.848006
{32, 25, 26, 27, 28, 29, 30, 31}
              Val
NotesID          
33       2.020679
34      -2.848006
